# Algorithm of Multi-AGVs Flow Optimisation"

author: "Emmanuel RIALLAND - ALBA INTELLIGENCE HONG KONG"

date: `j import Dates; Dates.Date(Dates.now())`


In [ ]:
# using Weave
# Weave.weave("AGV_markup.jl"; doctype="pandoc2pdf", cache=:refresh, pandoc_options=["--table-of-contents", "--number-sections", "--reference-links"])

using LightGraphs, SimpleWeightedGraphs

# [TODO]

- [TODO: Consider variants to A*]
- CHANGE ALL TO CartesianCoordinates


# NOTES

- Single planning (next task allocation): matrix 2D because only sorting by distance (cost = time) without considering collisions.

- Multiple planning: 3D

- $T_{Max}$ is currently fixed at 300s. It could just be a minimum to be dynamically increased if A\* search is unsuccessful.


In [ ]:
# HARDCODED IN SECONDS. ALL SPEEDS ARE MULTIPLES.
# THIS WAY: TURN = 1 UNIT. BCK = 2 UNITS. FWD = 4 UNITS.

Tmax = 5 * 60
TStep = 0.50
SliceMax = Int64(ceil(Tmax / TStep))

AGVLength = 4.0
AGVWidth = 3.0

3.0

# Assumptions

- Everything in SI.

- Time-step short enough to never have jump-overs even at full speed.

- AGV dimensions approximated in multiple of twice the grid so that all dimensions / tests can be done as centre location +/- value in integers.

- AGV dimensions include palettes.

- Should include security perimeter monitored by AGV?

- All standardised dimensions in floating point. All simulation dimensions in integers. Type mistakes are easier to pick up.

- Dimensions of the floor plan is in units of the gridsize.

- Once a palette is taken off a shelf, it cannot be put back (putting back might mess up warehouse software and really not obvious is a lot of time would actually be saved).
- Basically, any movement up or dowm to do anything will not be cancelled or replanned half-way through.
- The logic to track if any TPaletteLocation is occupied or not is ignored. We rely on the warehousing software to track that for us.


# Data structures

## Data types

### Location / Time

$TLocation$:

- $x, y:: Float64$ for standardised units

- $s = CartesianIndex(simx, simx::Int64)$ for simulation units


In [3]:
abstract type AbstractLocation end

struct TLocation <: AbstractLocation
    x::Float64
    y::Float64
    s::CartesianIndex{2}
end

$height$:

Represents how high a fork is at a particular time. This is necessary since any pick-up / delivery can be interrupted.

- $h:: Float64$ for standardised units

- $simh::Int64$ for simulation units


In [4]:
struct THeight
    h::Float64
    simh::Int64
end

$TTime$:

- $t:: Float64$ for standardised units

- $simt::Int64$ for simulation units


In [5]:
struct TTime
    t::Float64
    simt::Int64
end

### Direction of an AGV

$TDirection$::

- Enum: [Up, Down, Left, Right]


In [6]:
@enum TDirection begin
    Up
    Down
    Left
    Right
end

## Structures

### Parkings:

- Parking ID

- Parking location: $TLocation$


In [7]:
struct TParking <: AbstractLocation
    ID::String
    loc::TLocation
end

### Palette location (they do more than palettes!)

Palette locations is any spot that can have a palette!

- Conveyor or Delivery belt, Top or Bottom drop areas, Quay
- Which side of that location needs to be faced to pick up/drop
- NEED TO SEPARATE TIMES RELATED TO THE TASK PROVIDED BY THE QUEUING SYSTEM, VS. TIME CALCULATED FOR A GIVEN AGV MODEL.

- ID

- Location: $TLocation$

- How high a paletter is located:.level is an Integer that wil be = 0 for ground located

- Whether the location is currently occupied. Logic for change of occupancy NOT implemented (see Assumptions)


In [8]:
struct TPaletteLocation <: AbstractLocation
    ID::String

    loc::TLocation
    height::THeight

    sideToFace::TDirection
    isFull::Bool
end

### Tasks: Know at algo start

- Task ID ([TODO: Transform to data type])

- Location start ($s_j$): $TLocation$, $onrack$, $level$. $level$ only in on racks ($onrack == true$)

- Location target ($g_j$): $TLocation$, $onrack$, $level$. $level$ only in on racks ($onrack == true$)


In [9]:
struct TTask
    ID::String
    start::TPaletteLocation
    target::TPaletteLocation
end

### AGV:

- ID number: Known at algo start
- Initial position: $TLocation$ Known at algo start
- Current position: $TLocation$.
- Final position: $TLocation$.
- Final time stamp: $TTime$
- State:

  - $Direction$: $TDirection$
  - $forkHeight$: how high the fork currently is (in shelf simulation units!) since movements up or down can be interrupted at any time for replanning.
  - $isLoaded$: $Bool$
  - $isSideways$:: $Bool$ - is the AGV facing a shelf but otherwise ready to go (about to load or after loading, but no idea where to go).
  - [TODO: More TBD]

- List of tasks as list of tuples allocated to that AGV: Initialised empty

  - Task ID
  - time $TTime$ in simulation units from previous step to reach task. The list is updated at each iteration using the release time calculated by A\*. $t = 0$ at the start.

- Allocated parking . Full structure for the moment. (Maybe $Parking_{ID}$ later?).


In [ ]:
struct TAGV
    ID::String
    start::TLocation
    current::TLocation
    target::TLocation

    time::TTime

    direction::TDirection

    forkHeight::THeight
    isLoaded::Bool
    isSideways::Bool

    listTasks::AbstractVector{TTask}
    park::TParking
end

## Global variables

Precompute FloorPlan information

### Create list of obstacles


In [ ]:
function cFloorPlanObstacles(FloorPlan)
    nRow, nCol = size(FloorPlan)
    return [CartesianIndex(i, j) for i in 1:nRow,
            j in 1:nCol
            if FloorPlan[i, j] == 1]
end

function vFloorPlanObstacles(FloorPlan)
    nRow, nCol = size(FloorPlan)
    return [c2v(CartesianIndex(i, j), nCol) for i in 1:nRow,
            j in 1:nCol
            if FloorPlan[i, j] == 1]
end

vFloorPlanObstacles (generic function with 1 method)

### Task end to task end optimal times

- Initialise matrix $\tau$ with dimensions $length(ListTasks) \times length(ListTasks)$ Pre-calculate transfers from the end of one task to the end of another: $\tau_{i, j} = time\_2D(\mathbb{G}, g_i, g_j)$. That matrix is NOT symmetrical. It includes transfer time + and up/down fork movement. Each task ends with the AGV facing its pick-up/delivery point.


In [ ]:
function τ(G, listOfTasks, i, j)
    nTasks = length(listOfTasks)
    M = Matrix{TTime}(undef, (nTasks, nTasks))

    for i ∈ 1:nTasks
        for j ∈ 1:nTasks
            # Transfer time
            M[i, j] = time_2D(G, listOfTasks[i].target.loc,
                listOfTasks[j].target.loc).TTime
        end
    end

    return M
end

τ (generic function with 1 method)

### FloorPlan information structure


In [ ]:
struct FloorInfo
    plan
    nRow
    nCol
    obst_cart
    pbst_vert
end

function precompute_floorplan_info(FloorPlan)
    nRow, nCol = size(FloorPlan)

    return FloorInfo(
        plan=FloorPlan,
        nRow=nRow,
        nCol=nCol,
        obst_cart=cFloorPlanObstacles(FloorPlan),
        obst_vert=vFloorPlanObstacles(FloorPlan))
end

precompute_floorplan_info (generic function with 1 method)

# Utility Functions

## Reciprocal conversions between simulation coordinates and node numbers.

Coordinates are expressed in $CartesianIndex()$. Nodes are vertices expessed as $Int64$

### Conversions in 2 dimensions


In [ ]:
function c2v(p::CartesianIndex{2}, nCol::Integer)::Int64
    # Beware of 0 vs. 1 indexing
    return (p[1] - 1) * nCol + (p[2] - 1) + 1
end

function c2v(l::TLocation, nCol::Integer)::Int64
    return c2v(TLocation.s, nCol)
end

function c2v(p::TPaletteLocation, nCol::Integer)::Int64
    return c2v(p.loc, nCol)
end

function v2c(n::Integer, nCol::Integer)::CartesianIndex{2}
    # Beware of 0 vs. 1 indexing
    x = 1 + div(n - 1, nCol)
    y = 1 + rem(n - 1, nCol)
    return CartesianIndex(x, y)
end

v2c (generic function with 1 method)

### Conversions in 3 dimensions

3 dimensions is the 2 (x, y) plus time


In [ ]:
function c2v(p::CartesianIndex{3},
    nCol::Integer, nTime::Integer)::Int64
    # Beware of 0 vs. 1 indexing
    return ((p[1] - 1) * nCol + (p[2] - 1)) * nTime + (p[3] - 1) + 1
end

function c2v(l::TLocation, time::TTime,
    nCol::Integer, nTime::Integer)::Int64
    x, y = l.s
    return c2v(CartesianIndex(x, y, time.simt), nCol, nTime)
end

function c2v(p::TPaletteLocation, time::TTime,
    nCol::Integer, nTime::Integer)::Int64
    return c2v(p.loc, time, nCol, nTime)
end

function v2c(v::Integer,
    nCol::Integer, nTime::Integer)::CartesianIndex{3}

    # Beware of 0 vs. 1 indexing
    v0 = v - 1

    xy = 1 + div(v0, nTime)
    x = 1 + div(xy - 1, nCol)
    y = 1 + rem(xy - 1, nCol)

    t = 1 + rem(v0, nTime)

    return CartesianIndex(x, y, t)
end

v2c (generic function with 2 methods)

## $create\_list\_of\_parkings(n)$

Create $n$ parking positions with:

- All values are in standardised dimensions

- Number of parkings located next to sources of tasks in proportion of coming orders. Assume that this being done, equal probability of use is acceptable (no priority in the order of filling the parking).

- All different

- At least one per AGV

- Ensure parking positions do not prevent any traffic once an AGV is occupying it.
  - WARNING: this is necessary to guarantee that searches will always find solution:


In [ ]:
function create_list_parkings(M)::AbstractVector{TParking}
    return []
end

## $scale\_to\_simulation()$

Starting for a floorplan, create a matrix where each position of the floorplan is converted into a square of equal content but broken down into simulation units
# %%

function scale_to_simulation(F::AbstractMatrix{Int64}, GridSize::Real)

    GridSimMultiplier = Int64(ceil(1.0 / GridSize))

    # Create an empty matrix that will contain the simulation scaled properly.
    nRow, nCol = size(F)

    nSimRow = GridSimMultiplier * nRow
    nSimCol = GridSimMultiplier * nCol

    M = zeros(Int64, (nSimRow, nSimCol))

    # For each location in the resulting matrix, change to the floorplan value.
    for i ∈ 1:nRow
        for j ∈ 1:nCol
            # Calculate the beginning- and end-rows to be changed
            i_beg =  1 + GridSimMultiplier * (i - 1)
            i_end =      GridSimMultiplier * i

            # ditto for columns
            j_beg =  1 + GridSimMultiplier * (j - 1)
            j_end =      GridSimMultiplier * j

            # Block change
            M[i_beg:i_end, j_beg:j_end] .= F[i, j]
        end
    end

    return M
end

LoadError: syntax: extra token "for" after end of expression

## $create\_task\_pairing\_times(listOfTasks)$

Precomputes the optimal time to go from one finished task to the end of another for any pairing of tasks.
This is not symmetrical and calculated both ways.
If only a single task, nothing to do!


In [ ]:
# [CHECK: do we need to include the path in each result?]
function create_task_pairing_times(
    G::SimpleDiGraph,
    listOfTasks::AbstractVector{TTask},
    nCol::Int64)::AbstractVector{TTime}

    nTasks = length(listOfTasks)

    # Result matrix
    M = Array{TTime}(undef, (nTasks, nTasks))

    # TODO: As list comprehension?
    if nTasks > 1
        for i ∈ 1:nTasks
            for j ∈ 1:nTasks
                # Tasks
                t1 = listOfTasks[i].target
                t2 = listOfTasks[j].target

                # nodes at the end of task i and at the end of task j
                n1 = c2v(t1, nCol)
                n2 = c2v(t2, nCol)

                M[i, j] = time_2D(G, n1, n2)
            end
        end
    end

    return M
end

create_task_pairing_times (generic function with 1 method)

## $scale\_to\_simulation(object)$

Generic function to rescale the floor plan or object locations from standardised to simulation


In [ ]:
# Convert any Float64 to Int64. Can be used for location or time
function scale_to_simulation(x::Float64, factor::Float64)::Int64
    return Int64(ceil(x / factor))
end

function scale_to_simulation(p::THeight, tStep)::TTime
    r = deepcopy(p)
    r.simh = scale_to_simulation(p.h, tStep)
    return r
end

function scale_to_simulation(t::TTime, tStep)::TTime
    r = deepcopy(t)
    r.simt = scale_to_simulation(t.t, tStep)
    return r
end

function scale_to_simulation(loc::TLocation, gridSize::Float64)::TLocation
    l = deepcopy(loc)
    simx = scale_to_simulation(loc.x, gridSize)
    simy = scale_to_simulation(loc.y, gridSize)
    l.s = CartesianIndex(simx, simy)
    return l
end

function scale_to_simulation(p::TParking, gridSize::Float64)::TParking
    l = deepcopy(p)
    l.loc = scale_to_simulation(p.loc, gridSize)
    return l
end

function scale_to_simulation(p::TPaletteLocation, gridSize::Float64)::TPalette
    l = deepcopy(p)
    l.loc = scale_to_simulation(p.loc, gridSize)
    l.height = scale_to_simulation(p.height, gridSize)
    return l
end

function scale_to_simulation(t::TTask, gridSize::Float64)::TTask
    l = deepcopy(t)
    l.start = scale_to_simulation(t.start, gridSize)
    l.target = scale_to_simulation(t.target, gridSize)
    return l
end

function scale_to_simulation(t::Vector{TTask}, gridSize::Float64)::Vector{TTask}
    l = deepcopy(t)
    return map(t -> scale_to_simulation(t, gridSize), l)
end

function scale_to_simulation(a::TAGV, gridSize::Float64, tStep::Float64)::TAGV
    l = deepcopy(a)
    l.start = scale_to_simulation(a.start, gridSize)
    l.current = scale_to_simulation(a.current, gridSize)
    l.target = scale_to_simulation(a.target, gridSize)
    l.forkHeight = scale_to_simulation(a.forkHeight, gridSize)
    l.time = scale_to_simulation(a.time, tStep)
    l.listTasks = scale_to_simulation(a.listTasks, gridSize)
    l.park = scale_to_simulation(a.park, gridSize)
    return l
end

scale_to_simulation (generic function with 9 methods)

## $scale\_to\_standardised(object)$

Generic function to rescale the floor plan or object locations from simulation to standardised


In [ ]:
# Convert any Float64 to Int64. Can be used for location or time
function scale_to_standardised(v::Int64, factor::Float64)::Float64
    return Float64(factor * Float64(v))
end

function scale_to_standardised(p::THeight, tStep)::TTime
    r = deepcopy(p)
    r.h = scale_to_standardised(p.simh, tStep)

    return r
end

function scale_to_standardised(t::TTime, tStep)::TTime
    r = deepcopy(t)
    r.t = scale_to_standardised(t.simt, tStep)
    return r
end

function scale_to_standardised(loc::TLocation, gridSize::Float64)::TLocation
    l = deepcopy(loc)
    l.x = scale_to_standardised(loc.s[1], gridSize)
    l.y = scale_to_standardised(loc.s[2], gridSize)
    return l
end

function scale_to_standardised(p::TParking, gridSize::Float64)::TParking
    l = deepcopy(p)
    l.loc = scale_to_standardised(p.loc, gridSize)
    return l
end

function scale_to_standardised(p::TPaletteLocation, gridSize::Float64)::TPalette
    l = deepcopy(p)
    l.height = scale_to_standardised(p.height, gridSize)
    return l
end

function scale_to_standardised(t::TTask, gridSize::Float64)::TTask
    l = deepcopy(t)
    l.start = scale_to_standardised(t.start, gridSize)
    l.target = scale_to_standardised(t.target, gridSize)
    return l
end

function scale_to_standardised(t::Vector{TTask}, gridSize::Float64)::Vector{TTask}
    l = deepcopy(t)
    return map(t -> scale_to_standardised(t, gridSize), l)
end

function scale_to_standardised(a::TAGV, gridSize::Float64, tStep::Float64)::TAGV
    l = deepcopy(a)
    l.start = scale_to_standardised(a.start, gridSize)
    l.current = scale_to_standardised(a.current, gridSize)
    l.target = scale_to_standardised(a.target, gridSize)
    l.forkHeight = scale_to_standardised(a.forkHeight, gridSize)
    l.time = scale_to_standardised(a.time, tStep)
    l.listTasks = scale_to_simulation(a.listTasks, gridSize)
    l.park = scale_to_standardised(a.park, gridSize)
    return l
end

scale_to_standardised (generic function with 9 methods)

# Algorithm

## Expected inputs

All inputs are in Standardised dimensions.

- $F$: _Required_ Floor plan matrix
- $ListAGV$: _Required_. Check list length >= 1
- $ListTasks = []$: _Required_. List length can be 0. Must include the list of tasks that are already allocated at the start of the simulation.
- $ListOfParking = create\_list\_of\_parkings(length(ListAGV))$: _Optional_ List of parking slots. Created if missing.
- Speeds: _Required_ forward, backward, turn


## Wrapping

First call point is $optimise\_standardised\_units()$.

### Timing and Sizing parameters

Estimate adequate time step $t_{step}$ to scaled matrix to $M$. [CHECK: Default to 200ms]

Max simulation time $T_{Max}$: This time will be used when planning a path. It has to be long enough so that, given any configuration, A* will find a path from any current position, to achieve any task and go to any parking position. Currently 300s (converted to simulation time using $t_{step}$ in ${Step}_{Max}$). Track A* failures to check if long enough.

${Step}_{Max} = T_{Max} / t_{step}$ is the depth of the 3D planning matrix.

$GridSize$:

- size of each simulation square. AGV dimensions = +/- 1/2 width, +/- 1/2 length.
- Ensures never jumps over even max speed.

### Scale everything to Simulation dimensions

Rescale all relevant values from Standardised to Simulation:

- Matrix of the full floor plan: $M = Rescale(F)$
- Same for $ListOfParkings = [Parking_i]$, $\alpha_i$, $s_j$, $g_j$ to be appropriately scaled given $t_{step}$.

### Call main algo in simulation units

Call $optimise\_simulation\_units()$

### Scale everything back to standardise units

Rescale all relevant values from Simulation to Standardised

### Return

Results are returned in Standardised units.


### Code


In [ ]:
function optimise_standardise_units(
    F::AbstractMatrix{Float64},
    listAGVs::AbstractVector{TAGV},
    listTasks::AbstractVector{TTask};
    speedFwd::Float64=2.0,
    speedBck::Float64=1.0,
    speedTrn::Float64=0.5,
    listParkings::AbstractVector{TParking}=create_list_of_parkings(length(listAGVs)))

    # 600 STEPS - MATRIX DEPTH
    StepMax = Int64(ceil(TMax / TStep))

    # GridSize = 50 cm
    GridSize = Float64(ceil(speedTrn / TStep))
    M = scale_to_simulation(F, GridSize)

    simAGVLength = Int64(ceil(AVGLength / GridSize))
    simAGVWidth = Int64(ceil(AVGWidth / GridSize))

    # ALERT: FINALISE RETURN FORMAT
    simulation_optim = optimise_standardise_units(
        M, StepMax,
        scale_to_simulation(listAGVs),
        scale_to_simulation(listTasks),
        scale_to_simulation(speedFwd),
        scale_to_simulation(speedBck),
        scale_to_simulation(speedTrn),
        scale_to_simulation(listParkings))
end

optimise_standardise_units (generic function with 1 method)

## $optimise\_simulation\_units()$ - Preparation of the Main Loop

### Preamble

Each AGV will have:

- a list of tasks with a list of release time.

(BUSY WITH TASK --- RELEASE TIME)+ --- GO TO PARKING

The number of busy times can be 0. Each AGV has a final release time.

### Initialisation

- Using $M$, precompute 2D graph $\mathbb{G}$.


In [ ]:
function floorplan_to_2Dgraph(FloorPlan)
    nRow, nCol = size(FloorPlan)
    graph = SimpleWeightedGraph(nRow * nCol)

    for row in 1:nRow, col in 1:nCol,
        p in next_moves_2D(CartesianIndex(row, col), nRow, nCol)

        origin = CartesianIndex(row, col)
        target = p

        cObstacles = cFloorPlanObstacles(FloorPlan)

        # [TODO: To be extracted, generalised, cObstables pre-computed.]
        hcost = (target in cObstacles || origin in cObstacles) ? 100 : 1
        add_edge!(graph, c2v(origin, nCol), c2v(target, nCol), hcost)
    end

    return graph
end

floorplan_to_2Dgraph (generic function with 1 method)

- Using $M$, precompute $M_0$ with $T_{Max}$ slices.


In [ ]:
function pre_computed_3DMatrix(
    M::AbstractMatrix{Int64},
    nSlices::Int64)::AbstractArray{Int64}

    return repeat(M, nSlices)
end

pre_computed_3DMatrix (generic function with 1 method)

- For each AGV, if it already has a task, calculate its completion time $time\ 2D(\mathbb{G}, \alpha_i, g_j)$. This is the starting point of the planning loops.

- Initialise the list of tasks of each AGV to just initial position $PlanningList = [\alpha_i=[\alpha_{i, 0}]]$. Each $\alpha_i$ will grow with a list of tasks $\alpha_i = [\alpha_{i, 0}, \tau_{i, 1}, , \tau_{i, 2}, ...]$

- $PlanningList = [\alpha_i]$ will later contain a list of AGVs $\alpha_i$ each containing their allocated tasks (if any). It does not contain the list of parking locations which may vary from iteration to iteration.


## Start Main Loop


In [ ]:
function optimise_simulation_units(
    M::AbstractMatrix{Int64},
    StepMax::Int64,
    listAGVs::AbstractVector{TAGV},
    listTasks::AbstractVector{TTask},
    speedFwd::Int64,
    speedBck::Int64,
    speedTrn::Int64,
    listParkings::AbstractVector{TParking})

    floorGraph = pre_computed_2Dgraph(M)
    M0 = pre_computed_3DMatrix(M, StepMax)


    unAllocatedTasks = deepcopy(listTasks)

    while true

        for current_task ∈ unAllocatedTasks

            # Start simulation at t = 0
            t::Int64 = 0
        end

        # Replicate a Repeat-Until loop
        isempty(unAllocatedTasks) && break
    end

    # ALERT: FINALISE RETURN FORMAT
    return Nothing
end

optimise_simulation_units (generic function with 1 method)

$UnAllocatedTasks = ListTasks$ (That is the initial list of tasks)

**REPEAT** ------------------------------------------------------------------------------

### Optimal time for the best AGV/Task pair

Planning time starts at time $t::Int64=0$.

#### Assuming some tasks are not allocated yet

$ListTimePairings = []$ will contain the list of times: time(AGV $\alpha_i$, unallocated task $\tau_j$)

- For each Task $\tau_j$ in $UnAllocatedTasks != []$ (if $UnAllocatedTasks[]$ is not empty):

  - For each $\alpha_i$:

    - Take the time of where the AGV is: get the final position $current_i$ of $\alpha_i$, where $position_i$ is its initial position for the first iteration (if no task), or the position of its final release location $g_j$ given its list of tasks.

    - Take the time to complete the new tasks: $offset_i = time\_2D(\mathbb{G}, \alpha_i, g_i)$ when the list of tasks is empty, $offset_i = \tau_{i, j}$ for $i$ being the last task in $\alpha_i$.

    - Calculate $time(\alpha_i, g_j) = current_i + offset_i$

    - Push the result (with all relevant information) into $ListTimePairings$

#### Sort

$ListTimePairings$ contains all the current $\alpha_i$ with each of them has been allocated a single new task. $ListTimePairings$ may be $[]$.

- Sort all times in $ListTimePairings$ in increasing order and find the shortest. This is the only pair AGV / Task (if any) that will be added to the planning..

- Add the task to that AGV with a time which is the one stored in $ListTimePairings$. Take the previous $PlanningList$ and replace the $\alpha_i$ which has a new task.

- Reorder all the AGVs in decreasing order of total (including updated) release time. Store into $PlanningList$.

  - By construction, that list will contain all the tasks which were previously allocated + only one additional task.

  - The list should only contain entries where there is a new task added to already existing $\alpha_i$. In other words, choosing an entry will always guarantee that only a single new task is added to the Planning List.

  - We now have a new list of $\alpha_i=[\alpha_{i, 0}, \tau_{i, 1}, , \tau_{i, 2}, ...]$ where AT MOST ONE of them has an additional task. This is the list to be sorted in decreasing order of total time (total release time + time to achieve new task). ONLY ONE if a task was available or NOTHING if the list of tasks was empty to start with.

- Remove the newly allocated task from $UnAllocatedTasks[]$.

### Planning Loop

$ListAllocationParking = []$: To store all parking allocation as it happens one by one.

$PlanningListWithParkings$: To store all the $\alpha_i$ with their respective parking before planning.

#### Allocate parking locations

Given the list of tasks and the parking position already allocated in $ListAllocationParking$, create list of remaining parking positions accounting.

Looping on the AGVs, allocate to each $\alpha_i$ the closest parking $Parking_i$ from its last $\tau_{i, j}$ in its list of tasks. Push each into $PlanningListWithParkings$.

Sort $PlanningListWithParkings$ in decreasing time order.

#### Start planning loop

$ListFullPath = []$: To store all paths one by one.

**FOR** ------------

For each $\alpha_i \in PlanningList$:

- $M_i = M_{i-1}$ (Note that $M_0$ is precalculated)

- Add slices to $M_i$ so that at least $T_{Max}$ slices of buffer.

**Create detailed plan for $Path_i$**

- Reset the clock of the AGV: $t_i = 0$

- Plan $PlanningListWithParkings[i]$ on $M_i$. The planning must record the times of the final realease time at which all tasks for that $\alpha_i$ are completed [CHECK: Is the time of parking to be recorded as well]. The result is $FullPath$. If not solution, raise error to increase $T_{Max}$.
- Push $FullPath$ into $ListFullPath$

- Push found path $FullPath$ into $M_i$ (i.e. obstruct that path).

- **NEXT**

**UNTIL ** All tasks have been planned

$optimise\_simulation\_units$: Return list of $\alpha_i=[\alpha_{i, 0}, \tau_{i, 1}, , \tau_{i, 2}, ...]$, and $ListFullPath$


---

# LOST BLOCK?

$\tau()$ matrix can be done on 2D.
TODO: However, the graph needs to be from 3D matrix without path back.

- from the 3D matrix
- without path back
- provide turns. Means 2D for location + 1D (4 values) for direction + 1D for time.
- when generating 'next_moves_2D' needs also a counter reflecting how long an AGV stays in a loction going up / down, how long to turn once turning has been decided.
- the changes from one node to the other is therefore highly dependent on AGV, therefore need to regenerate full matrix + graph for each path search.

---


# Path search

## $time\_2D(\mathbb{G}, a, b)$

Time the optimal paths in 2D from location $a$ to $b$ in a precomputed graph $\mathbb{G}$ using Dijkstra.

Returns full path and total execution time.

All dimensions are in simulation dimensions.


In [ ]:
function time_2D(G::SimpleWeightedGraph,
    start_v::Integer, exit_v::Integer)
    return enumerate_paths(dijkstra_shortest_paths(G, start_v), exit_v)
end

function time_2D(G::SimpleWeightedGraph,
    start::CartesianIndex{2}, exit::CartesianIndex{2},
    nCol::Integer)
    start_v = c2v(start, nCol)
    exit_v = c2v(exit, nCol)
    return time_2D(G, start_v, exit_v)
end

function time_2D(G::SimpleWeightedGraph,
    start::TLocation, exit::TLocation,
    nCol::Integer)
    start_v = c2v(start, nCol)
    exit_v = c2v(exit, nCol)
    return time_2D(G, start_v, exit_v)
end

time_2D (generic function with 3 methods)

## time3D

Note: end point is a line in 3D. To arrive as early as possible means penalising delays. Transitions through time located at end point should be 0 when staying anywhere else costs time.


## Move generator


In [ ]:
# Returns a list of Cartesian indices for each of the possible moves from a
# given position  = Move generator.
function next_moves_2D(p::CartesianIndex{2}, nRow, nCol)
    x = p[1]
    y = p[2]

    down = x > 1 ? x - 1 : x
    up = x < nRow ? x + 1 : x
    left = y > 1 ? y - 1 : y
    right = y < nCol ? y + 1 : y
    [CartesianIndex(x, y) for x in down:up, y in left:right]
end

# Given a list of vertices, return the coodinates of the path steps
v2c_path(path, nCol) = [v2c(v, nCol) for v in path]


# start and exit are CartesianIndex()
function path_a_b(FloorPlan, start::CartesianIndex{2}, exit::CartesianIndex{2})

    nRow, nCol = size(FloorPlan)

    #########################################################################
    # Cost of a given path.
    # path[1] is the starting point
    vObstacles = vFloorPlanObstacles(FloorPlan)

    #########################################################################
    # Generate graph:
    #   iterate through each position
    #   create an edge from there to each successive move (N, S, E, W).
    #   move into obstacles are penalised with 100, possible moves cost 1.
    #
    # Missing nodes are created automatically and not used?

    G = floorplan_to_graph(FloorPlan)

    # Path is from top left to bottom right
    path = time_2D(G, start, exit, nCol)
    path_time = sum(map(v -> (v in vObstacles ? 100 : 1), path[2:end]))

    println("Solution has cost $(path_time):\n", v2c_path(path, nCol))

    return (v2c_path(path, nCol), path_time)
end

path_a_b (generic function with 1 method)

    Updating registry at `~/.julia/registries/General`
    Updating git-repo `https://github.com/JuliaRegistries/General.git`
   Resolving package versions...
    Updating `~/Development/_langs/julia/site-storage/environments/v1.6/Project.toml`
  [093fc24a] + LightGraphs v1.3.5
  No Changes to `~/Development/_langs/julia/site-storage/environments/v1.6/Manifest.toml`

In [ ]:
# IMPORTANT:
# 0 - available
# 1 - blocked
FloorPlan = [
    0 0 0 0 0;
    1 0 1 0 1;
    1 0 1 0 1;
    1 0 1 0 1;
    1 0 1 0 1;
    1 0 1 0 1;
    1 0 1 0 1;
    1 0 1 0 1;
    1 0 1 0 1;
    0 0 0 0 0]

nRow = size(FloorPlan)[1]
nCol = size(FloorPlan)[2]

nBots = 2

2

In [ ]:
floorG = SimpleDiGraph(nRow * nCol)

# Create all horizontal edges
for r ∈ 1:nRow
    for c ∈ 1:(nCol-1)
        if (FloorPlan[r, c] == 0) && (FloorPlan[r, c+1] == 0)
            add_edge!(floorG, (r - 1) * nCol + c, (r - 1) * nCol + c + 1)
        end
    end
end

# Create all vertical edges
for c ∈ 1:nCol
    for r ∈ 1:(nRow-1)
        if (FloorPlan[r, c] == 0) && (FloorPlan[r+1, c] == 0)
            add_edge!(floorG, (r - 1) * nCol + c, (r - 1 + 1) * nCol + c)
        end
    end
end